In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import astropy.units as au
import astropy.constants as ca

In [2]:
#Semi empirical model + error for spectrum TRAPPIST-1 from Wilson et al 2021
trappist_SED = np.genfromtxt("../data/TRAPPIST1_Semimodel_Wilson/TRAPPIST-1_model_const_res_v07.ecsv", \
                              delimiter = "")
#unit erg A^-1 s^-1 cm^-2 

In [3]:
flux_TRAP = trappist_SED[:,1]  * au.erg * au.cm**(-2) * au.s**(-1) * au.Angstrom**(-1)
wavelength_TRAP = trappist_SED[:,0]  * au.Angstrom
#error of semiemprical model is 5% Wilson et al 2021
fluxerror_TRAP = (0.05*trappist_SED[:,1])  * au.erg * au.cm**(-2) * au.s**(-1) * au.Angstrom**(-1)

In [4]:
#Muscat Reaponse
muscat = pd.read_csv("../data/MuSCAT2_response/MuSCAT2_response.csv")

muscat = muscat.fillna(0) 
muscat["lamb"] = muscat.wavelength*1e1   #Angstrom

In [5]:
# Gaia response function for G filter
gaia = np.genfromtxt("../data/Gaia_response/GaiaDR2_Passbands.dat")
mask = gaia == 99.99 
gaia[mask] = 0
gaia[:,0] = gaia[:,0]*1e1  #Angstrom

In [6]:
#Interplotation to TRAPPIST-1 Gaia
rres = np.interp(gaia[:,0], wavelength_TRAP.value, flux_TRAP.value) 

In [7]:
#total flux in gaia passbands
gaiaflux  = np.trapz(gaia[:,1]*rres, x=gaia[:,0])

In [8]:
#normalisation of MM SED
f1 = flux_TRAP/gaiaflux

In [9]:
#Interpolation to TRAPPIST-1 M2
rres1 = np.interp(muscat.lamb, wavelength_TRAP.value, f1.value) 

In [10]:
#Wilson et al 2021 take 50% of the flux values as error
muscatflux  = np.asarray([np.trapz(muscat.Total_g*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_r*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_i*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_z*rres1, x=muscat.lamb) ])

In [11]:
muscatflux

array([0.02471587, 0.0544569 , 0.41515412, 0.5981133 ])

In [12]:
#Wilson et al 2021 take 50% of the flux values as error
muscatfluxerror  = np.asarray([np.trapz(muscat.Total_g*0.05*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_r*0.05*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_i*0.05*rres1, x=muscat.lamb), \
                   np.trapz(muscat.Total_z*0.05*rres1, x=muscat.lamb) ])

In [13]:
#Lienhardt et al 2020
dist_TRAP = 3.835084e+19 *au.cm 
dist_TRAP_error = 0.02 * 3.086e+18 *au.cm

lumi_TRAP = muscatflux*au.erg*au.cm**(-2)*au.s**(-1)*(4*np.pi*(dist_TRAP)**2)
#lumierror_TRAP = np.sqrt(4*np.pi*(dist_TRAP.value)**2 * brightnesserror_TRAP.value**2 + 8*np.pi*brightness_TRAP.value*dist_TRAP.value*dist_TRAP_error.value**2 ) *au.erg * au.s**(-1)
lumi_TRAP.to("W")


<Quantity [4.56809851e+31, 1.00649714e+32, 7.67306677e+32, 1.10546013e+33] W>

In [14]:
#safe passbandflux
with open("../results/values/brightness_TRAPPIST-1_withCCDeffeciency.txt", "w+") as f:
    f.write("g , r , i, z, units: [erg cm-2 s-1]  \n")
    f.write(str(muscatflux[0])+ ',' +  str(muscatflux[1])+ ',' + \
            str(muscatflux[2])+ ',' + str(muscatflux[3]) + '\n' )
    f.write(str(muscatfluxerror[0])+ ',' +  str(muscatfluxerror[1])+ ',' + \
            str(muscatfluxerror[2])+ ',' + str(muscatfluxerror[2]) + '\n' )

In [15]:
#safe lumi
with open("../results/values/lumi_TRAPPIST-1_withCCDeffeciency.txt", "w+") as f:
    f.write("g , r , i, z, units: [erg s-1] \n")
    f.write(str(lumi_TRAP[0].value)+ ',' +  str(lumi_TRAP[1].value)+ ',' + \
            str(lumi_TRAP[2].value)+ ',' + str(lumi_TRAP[3].value) )

# ------------------------------- Not needed at the moment ----------------------------

In [11]:
#Mean flux in muscat passbands
muscatflux_mean  = np.asarray([np.mean(muscat.Total_g*rres), \
                                np.mean(muscat.Total_r*rres), \
                                np.mean(muscat.Total_i*rres), \
                                np.mean(muscat.Total_z*rres)])

muscatfluxerror_mean  = np.asarray([np.mean(muscat.Total_g*0.5*rres), \
                   np.mean(muscat.Total_r*0.5*rres), \
                   np.mean(muscat.Total_i*0.5*rres), \
                   np.mean(muscat.Total_z*0.5*rres) ])

with open("../results/values/brightness_mean_TRAPPIST-1_withCCDeffeciency.txt", "w+") as f:
    f.write("g , r , i, z, units: [erg cm-2 s-1]  \n")
    f.write(str(muscatflux_mean[0])+ ',' +  str(muscatflux_mean[1])+ ',' + \
            str(muscatflux_mean[2])+ ',' + str(muscatflux_mean[3]) + '\n' )
    f.write(str(muscatfluxerror_mean[0])+ ',' +  str(muscatfluxerror_mean[1])+ ',' + \
            str(muscatfluxerror_mean[2])+ ',' + str(muscatfluxerror_mean[2]) + '\n' )

In [ ]:
### Notes: 

'''

Review uncertainties 

Calculation correction: At the moment I am using only the passbands but not the M2 effeciency of CCDs.

Possible correction: Concolution of flux_TRAP * filtertransmission_total 
--> Ekaterina did this in here study, so there you can learn how it should work. 


# Interpolate response from rwav to w:
rres = np.interp(tess.lambd, bt.lambd, bt.flux, left=0, right=0)

# Integrating the flux of the thermal
# spectrum times the response curve over wavelength:
tessflux = np.trapz(tess.response * rres, x=tess.lambd)*u.erg * u.cm**(-2) * u.s**(-1)

# literature values from White et al. 2015
r, rerr = 0.75 * R_sun, 0.03 * R_sun 

# TESS projected surface luminosity
tesslum = np.pi * r**2 * tessflux

# convert to astro units
tesslum = tesslum.to("erg /s")
tesslum

'''